In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms as T
import numpy as np

import nrrd
import pandas as pd
import os
import sys

import plotly.express as px

# sys.path.append("/mnt/raid/C1_ML_Analysis/source/autoencoder/src")
sys.path.append("/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/")
sys.path.append("/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/dl/")
sys.path.append("/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/dl/nets")


from dl.nets.layers import TimeDistributed
import dl.nets.us_simulation_jit as us_simulation_jit
from dl.nets.us_simu import VolumeSamplingBlindSweep


In [ ]:
mount_point = "/mnt/raid/C1_ML_Analysis/"
diffusor_np, diffusor_head = nrrd.read(os.path.join(mount_point, "simulated_data_export/placenta/FAM-025-0869-1_label11_resampled.nrrd"))

diffusor_size = diffusor_head['sizes']
diffusor_spacing = np.diag(diffusor_head['space directions'])

diffusor_origin = np.flip(diffusor_head['space origin'], axis=0)
diffusor_end = diffusor_origin + diffusor_spacing * diffusor_size

In [ ]:
diffusor_t = torch.tensor(diffusor_np.astype(float)).unsqueeze(0).unsqueeze(0).cuda()
diffusor_origin = torch.tensor(diffusor_origin.copy()).unsqueeze(0).cuda()
diffusor_end = torch.tensor(diffusor_end.copy()).unsqueeze(0).cuda()


In [ ]:

us_simulator = us_simulation_jit.MergedLinearLabel11().eval().cuda()

grid, inverse_grid, mask_fan = us_simulator.init_grids(256, 256, 128.0, -30.0, 20.0, 215.0, 0.7853981633974483)
grid = grid.cuda()
inverse_grid = inverse_grid.cuda()
mask_fan = mask_fan.cuda()

us_simulator_td = TimeDistributed(us_simulator, time_dim=2).eval()

vs = VolumeSamplingBlindSweep(mount_point=mount_point).cuda()

In [ ]:
vs.tags

In [ ]:
sweep = vs.get_sweep(diffusor_t, diffusor_origin, diffusor_end, vs.tags[0], use_random=False, simulator=us_simulator_td, grid=grid, inverse_grid=inverse_grid, mask_fan=mask_fan)

In [ ]:
fig = px.imshow(sweep.squeeze().detach().cpu().numpy(), animation_frame=0, binary_string=True)
fig.show()